In [25]:
import pandas as pd
import numpy as np
import time

df = pd.read_csv('puzzle_batches/small_batch.csv')
df = df[['FEN','Moves','Themes']].dropna()

In [26]:
# Execute computer move

import chess

for row in df.itertuples():
    board = chess.Board(row.FEN)

    computer_move = row.Moves.split()[0]
    board.push_uci(computer_move)
    df.loc[row.Index].FEN = board.fen()

df.drop(['Moves'], inplace=True, axis=1)

In [27]:
# Create valid_themes columns

start = time.time()

used_themes = {
    "advancedPawn", "advantage", "bishopEndgame", "capturingDefender", "crushing", "equality", "kingsideAttack", "defensiveMove", "endgame",
    "exposedKing", "fork", "hangingPiece", "interference", "knightEndgame", "middlegame", "opening", "pawnEndgame", "pin", "promotion", "queenEndgame",
    "queenRookEndgame", "queensideAttack", "rookEndgame", "skewer", "trappedPiece", "underPromotion", "xRayAttack", "zugzwang",
}

df['valid_themes'] = ""
for row in df.itertuples():
    row_themes = set(row.Themes.split())
    valid_themes = row_themes.intersection(used_themes)
    #print(f"valid themes {valid_themes}")

    df.loc[row.Index].valid_themes = " ".join(valid_themes)

themes_dummies = df.valid_themes.str.get_dummies(' ').add_prefix('theme_')
df = pd.concat([df, themes_dummies], axis=1)

df.drop(['Themes', 'valid_themes'], inplace=True, axis=1)
end = time.time()
print(end - start)

0.009802103042602539


In [28]:
# Create board positions with piece vectors
start = time.time()

squares = chess.SQUARE_NAMES

# pawn, knight, bishop, rook, king
# queen will be encoded as bishop + rook
pieces = ['p', 'n', 'b', 'r', 'k'] 
for s in squares:
    for p in pieces:
        df[f"{s}.{p}"] = 0

end = time.time()
print(end - start)

0.09100842475891113


In [29]:
# Transform FEN into board positions and pieces
start = time.time()

for row in df.itertuples():
    board = chess.Board(row.FEN)

    # color -> false = preto / true = branco
    for pos, piece in board.piece_map().items():
        square_name = chess.square_name(pos)
        piece_name = piece.symbol().lower()

        if piece_name == "q":
            df.at[row.Index, f"{square_name}.r"] = 1 if piece.color else -1
            df.at[row.Index, f"{square_name}.b"] = 1 if piece.color else -1
        else:
            df.at[row.Index, f"{square_name}.{piece_name}"] = 1 if piece.color else -1

end = time.time()
print(end - start)

0.027904033660888672


In [30]:
# Reorder columns
start = time.time()

for row in df.itertuples():
    board = chess.Board(row.FEN)
    df.at[row.Index, "color_to_play"] = int(board.turn) or -1
    df.at[row.Index, "castling_rights"] = int(board.castling_rights)


df.drop(['FEN'], inplace=True, axis=1)
end = time.time()
print(end - start)

0
0
r . . q . . . k
p p p . . . p B
. . . p . . . B
. . . . . . . .
. . . b . . . .
. . N . . P . b
P n Q . . P . .
R . . . R . K .
1
0
. . . . . . . .
p . . n . . p k
. . . p . p . p
. . . . p P . B
. . P p P . . .
P . . P . Q R P
. r . q . . P K
. . . . . . . .
1
0
r . r . . . k .
p . . . . p p p
. . . . p . . .
. . . . . . . .
. . . P . . . .
. . . . P . . .
q P R . . P P P
. . Q . . . K .
1
0
. r . . . . k .
. q . . . p . .
. p . P . . p .
p . p Q . . . p
. . P . . . . P
. R . . . . P .
. . . . . P K .
. . . . . . . .
1
0
. . . . . . . .
r . . . k . . p
. . p . p . p .
. r P . B . . .
p . . . . R . .
. . . . P . P P
P P . . . . . .
. . . . K . . .
0
0
. . k . . . r .
p . . . . . . .
. . P . . . . .
. . . p . b . p
. . . B . . P q
. P . . P . . P
P . . . . . . Q
. . R . . K . .
1
0
. . . . . . k .
. . . . . . p .
. R . . . . P .
. . . . r . . .
. . b . . . . .
. . . . . . . .
P . . . . K . .
. . . . . . . .
1
0
r . . . r . k .
. . . . . p p p
. . . . . . . .
. q . p . Q . .
p . p . 